# City similarity [ Bangkok - Kyoto ]

## Import zone

In [1]:
import gensim
import os
import collections
import smart_open
import random
import pickle
import pandas as pd
import copy
import re

C:\Users\sociocom\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


## Parameter zone

In [2]:
path_play = r'C:\Users\sociocom\Desktop\Crawler\backup\playing'

In [3]:
grid_size = 15
diff = 0.1/grid_size

In [94]:
bkk_lat = 13.685780
bkk_long = 100.484605
diff_wid = 0.1/grid_size

In [258]:
kyt_lat = 34.955205 #35.005205
kyt_long = 135.675300 #135.7353
diff_wid = (135.795300 - 135.675300)/grid_size
#135.675300,34.955205,135.795300,35.055205

In [259]:
#merge to 10 color
color_idx = {}
idx_color = {}

In [260]:
boundary = 10

## Function zone

In [5]:
def save_obj(file, obj):
    with open(file, 'wb') as f:
        pickle.dump(obj, f)

def load_obj(file):
    with open(file, 'rb') as f:
        return pickle.load(f)

In [6]:
def clean_place(df, name):
    return df[df['place']!=name]

In [7]:
def separate_loc(df, flat, flong):
    df['ex_lat'] = df['ex_lat'].apply(lambda x: float(x))
    df['ex_long'] = df['ex_long'].apply(lambda x: float(x))
    data = []
    lat = flat
    long = flong
    for i in range(grid_size):
        for j in range(grid_size):
            data.append(df[(df['ex_lat'] >= lat) & (df['ex_lat'] < lat+diff) & (df['ex_long'] >= long) & (df['ex_long'] < long + diff_wid)])
            idx = []
            idx.append((long, lat))
            idx.append((long+diff_wid, lat))
            #print(lat," ",long)
            lat += diff
            idx.append((long, lat))
            idx.append((long+diff_wid, lat))
            u = i*grid_size + j
            idx.append(str(u))
            idx_color[str(u)] = idx
        long += diff_wid
        lat = flat
    return data

In [8]:
def read_corpus(fname, tokens_only=False):
    with smart_open.smart_open(fname, encoding="utf-8") as f:
        for i, line in enumerate(f):
            if tokens_only:
                yield gensim.utils.simple_preprocess(line)
            else:
                # For training data, add tags
                yield gensim.models.doc2vec.TaggedDocument(gensim.utils.simple_preprocess(line), [i])

In [9]:
def in_tuple(tp, x):
    return [z for y,z in tp if y == x]

In [10]:
parent = {}


In [11]:
#new
def merge_area(threshold, idx_color,city_list):
    for i in range(len(idx_color)):
        e = i+1 if ((i+1)%grid_size != 0) and (i+1 < len(idx_color)) else -1
        s = i+grid_size if i+grid_size < len(idx_color) else -1
        if e>0:
            if (len(city_list[i])+len(city_list[e])) > 0:
                similare = (in_tuple(all_sims[i], e)[0] + in_tuple(all_sims[e], i)[0])/(2*(len(city_list[i])+len(city_list[e])))
                if similare > threshold:
                    idx_color[str(e)][4] = idx_color[str(i)][4]
            else:
                idx_color[str(e)][4] = idx_color[str(i)][4]
        if s>0:
            if (len(city_list[i])+len(city_list[s])) > 0:
                similars = (in_tuple(all_sims[i], s)[0] + in_tuple(all_sims[s], i)[0])/(2*(len(city_list[i])+len(city_list[s])))
                if similars > threshold:
                    idx_color[str(s)][4] = idx_color[str(i)][4]
            else:
                idx_color[str(s)][4] = idx_color[str(i)][4]
    return idx_color

In [15]:
#new no cardinal
def merge_area(threshold, idx_color,city_list):
    for i in range(len(idx_color)):
        e = i+1 if ((i+1)%grid_size != 0) and (i+1 < len(idx_color)) else -1
        s = i+grid_size if i+grid_size < len(idx_color) else -1
        if e>0:
            if (len(city_list[i])+len(city_list[e])) > 0:
                similare = (in_tuple(all_sims[i], e)[0] + in_tuple(all_sims[e], i)[0])/(2)
                if similare > threshold:
                    idx_color[str(e)][4] = idx_color[str(i)][4]
            else:
                idx_color[str(e)][4] = idx_color[str(i)][4]
        if s>0:
            if (len(city_list[i])+len(city_list[s])) > 0:
                similars = (in_tuple(all_sims[i], s)[0] + in_tuple(all_sims[s], i)[0])/(2)
                if similars > threshold:
                    idx_color[str(s)][4] = idx_color[str(i)][4]
            else:
                idx_color[str(s)][4] = idx_color[str(i)][4]
    return idx_color

In [16]:
#old
def merge_area(threshold, idx_color,city_list):
    for i in range(len(idx_color)):
        w = i-1 if (i-1)%grid_size != (grid_size-1) else -1
        e = i+1 if ((i+1)%grid_size != 0) and (i+1 < len(idx_color)) else -1
        n = i-grid_size
        s = i+grid_size if i+grid_size < len(idx_color) else -1
        if len(city_list[i])<boundary:
            idx_color[str(i)][4] = "-1"
        else:
            if w>0 and len(city_list[w])>=boundary:
                similarw = (in_tuple(all_sims[i], w)[0] + in_tuple(all_sims[w], i)[0])/(2*(len(city_list[i])+len(city_list[w])))
                if similarw > threshold:
                    idx_color[str(w)][4] = idx_color[str(i)][4]
            if n>0 and len(city_list[n])>=boundary:
                similarn = (in_tuple(all_sims[i], n)[0] + in_tuple(all_sims[n], i)[0])/(2*(len(city_list[i])+len(city_list[n])))
                if similarn > threshold:
                    idx_color[str(n)][4] = idx_color[str(i)][4]
            if e>0 and len(city_list[e])>=boundary:
                similare = (in_tuple(all_sims[i], e)[0] + in_tuple(all_sims[e], i)[0])/(2*(len(city_list[i])+len(city_list[e])))
                if similare > threshold:
                    idx_color[str(e)][4] = idx_color[str(i)][4]
            if s>0 and len(city_list[s])>=boundary:
                similars = (in_tuple(all_sims[i], s)[0] + in_tuple(all_sims[s], i)[0])/(2*(len(city_list[i])+len(city_list[s])))
                if similars > threshold:
                    idx_color[str(s)][4] = idx_color[str(i)][4]
    return idx_color

In [17]:
def count_areas(idx_color):
    areas = set()
    for i in range(grid_size):
        for j in range(grid_size):
            areas.add(idx_color[str((i*grid_size)+j)][4])
    return len(areas)

In [18]:
def print_areas(idx_color):
    areas = set()
    for i in range(grid_size):
        s = ""
        for j in range(grid_size):
            idx = str((i*grid_size)+j)
            if int(idx_color[idx][4]) < grid_size and int(idx_color[idx][4]) >= 0:
                s += " "
            s += str(idx_color[idx][4]) +" "
            areas.add(idx_color[idx][4])
        print(s)
    print("\n #areas :" , len(areas))
    return areas

In [19]:
def print_volumn(list_place):
    for i in range(grid_size):
        s = ""
        for j in range(grid_size):
            idx = (i*10)+j
            if len(list_place[idx]) < 99 and len(list_place[idx]) >= 0:
                s += " "
                if len(list_place[idx]) <10:
                    s+= " "
            s += str(len(list_place[idx])) +" "
        print(s)

In [20]:
#no
def clean_reply(df):
    for i in range(len(df)):
        if (df["ex_lat"][i] == -1) | (df["tweet_reply_username"][i] == -1) | (df["tweet_reply_username"][i] == None):
            continue
        idx_reply = df["text"][i].find(df["tweet_reply_username"][i])-1
        leng = len(df["tweet_reply_username"][i])+1
        df.at[i, "text"] = df["text"][i][:idx_reply] + " <MENTION> " + df["text"][i][idx_reply+leng:]

In [21]:
#no
def clean_url(city):
    city_str = []
    for y in city:
        doc = ""
        for x in y['text']:
            print(x)
            if "https://" in x.lower():
                idx = (x.lower().find("https://"))-1
                x = x[:idx] + " <URL>"
            if "I'm at " in x and "<URL>" in x:
                idx_i = (x.find("I'm at "))
                idx_url = (x.find("<URL>"))+5
                x = x[:idx_i]+x[idx_url:]+"<CHECK_IN>"
            doc += x + ". "
        doc += " \n"
        city_str.append(doc)
        return city_str

## Data zone

## Kyoto 

In [261]:
df_kyt_old= load_obj(path_play + r"\kyoto_translated1")
df_kyt_old.rename(columns={"id_str":"tweet_id", "screen_name":"acc", "id_user_str":"acc_id", "created_at":"created_time",
                           "in_reply_to_status_id_str":"tweet_reply_id", "in_reply_to_user_id_str":"tweet_reply_user_id", "in_reply_to_screen_name":"tweet_reply_username"}, inplace=True)

In [262]:
list_df = []
for filename in os.listdir(path_play+"\\kyt"):
    with open(path_play + "\\kyt\\" + filename, "rb") as f:
        in_path = path_play + "\\kyt\\" + filename
        print(in_path)
        df = load_obj(in_path)
        df.reset_index(inplace=True, drop="index")
        df.drop(columns=["text","place"], inplace=True)
        df.rename(columns={"text_trans":"text", "place_trans":"place", "id_str":"tweet_id"}, inplace=True)
        list_df.append(df)

C:\Users\sociocom\Desktop\Crawler\backup\playing\kyt\15-22June_translated_kyt_tweet


In [263]:
df_kyt_new = pd.concat(list_df)

In [264]:
df_kyt = pd.concat([df_kyt_old, df_kyt_new])

In [265]:
df_kyt.describe()

,acc,acc_id,created_time,ex_lat,ex_long,lang,lot_ne,lot_sw,place,text,tweet_id,tweet_reply_id,tweet_reply_user_id,tweet_reply_username
count,70104,70104,70104,70104,70104,70104,70103,70103,70104,70104,70104,29279,29362,29362
unique,12712,12681,67274,10317,10194,2,285,285,317,69295,69992,10481,6051,6055
top,amds_kyotoKyoto,482601329,Mon Jun 04 20:14:48 +0000 2018,-1,-1,ja,"135.779805, 35.018685","135.730513, 34.99342",Kyoto City Nakagyo-ku,It shook again.,1003731817512538112,-1,-1,-1
freq,1262,1262,15,25753,25753,67439,16830,16830,15274,18,15,18625,18140,18140


In [266]:
df_kyt.drop_duplicates(inplace=True)

In [267]:
df_kyt = clean_place(df_kyt, "Japan")
df_kyt = clean_place(df_kyt, "Kyoto")

In [268]:
df_kyt.reset_index(drop="index", inplace=True)

In [269]:
df_kyt.rename(columns={"in_reply_to_screen_name":"tweet_reply_username"}, inplace=True)

In [270]:
#clean_reply(df_kyt)
for i in range(len(df_kyt)):
    if (df_kyt["ex_lat"][i] == -1) | (df_kyt["tweet_reply_username"][i] == -1) | (df_kyt["tweet_reply_username"][i] == None):
        continue
    idx_reply = df_kyt["text"][i].find(df_kyt["tweet_reply_username"][i])-2
    leng = len(df_kyt["tweet_reply_username"][i])+2
    df_kyt.at[i, "text"] = df_kyt["text"][i][:idx_reply] + " <MENTION> " + df_kyt["text"][i][idx_reply+leng:]

In [271]:
kyt = separate_loc(df_kyt, kyt_lat, kyt_long)

In [272]:
#kyt_str = clean_url(kyt)
kyt_str = []
for y in kyt:
    doc = ""
    for x in y['text']:
        if "https://" in x.lower():
            idx = (x.lower().find("https://"))-1
            x = x[:idx] + " <URL>"
        if ("I'm at " in x or "I 'm at " in x) and "<URL>" in x:
            idx_i = (x.find("I'm at "))
            if idx_i == -1:
                idx_i = (x.find("I 'm at "))
            idx_url = (x.find("<URL>"))+5
            x = x[:idx_i]+x[idx_url:]+"<CHECK_IN>"
        doc += x + ". "
    doc += " \n"
    kyt_str.append(doc)

In [273]:
f = open(path_play + "/kyt_demo_train3","w",encoding="utf-8")
#file = open(path + "bkk_demo_test","w",encoding="utf-8")
for x in kyt_str:
    f.write(x)
    print(x)
f.close()

 

<CHECK_IN>. <CHECK_IN>. Was it a little early? (@ FamilyMart Daido Tozai Machi in Kyoto City, Kyoto Prefecture) <URL>. <CHECK_IN>. <CHECK_IN>. <CHECK_IN>. <CHECK_IN>. <CHECK_IN>. <CHECK_IN>. <CHECK_IN>. <CHECK_IN>. <CHECK_IN>. <CHECK_IN>. Prescription and a bit of shopping (@ Ducks Rock West Central store in Kyoto City, Kyoto Prefecture) <URL>. <CHECK_IN>.  

<CHECK_IN>. <CHECK_IN>. <CHECK_IN>. <CHECK_IN>. <CHECK_IN>. Individual participation program = Individual participation game @ Kerubito Futsal ... <URL>. <CHECK_IN>. Individual participation program = Individual participation school @ Kerubito Futsal ... <URL>. Today, the third round ⚽ Location: Kerubito Futsal Club (Kelvit Futsal Club) <URL>. Futsal at the Katsura soccer club! I was sleeping at home and I was a little late but it was fun. Thanks everyone ~ ne ~ # Katsura Soccer Club # Futsal # Kicking the ball is still a fun place: ... <URL>. Ann (@ Seven Eleven Kyoto Otani Nakayama shop in Kyoto City, Kyoto Prefecture) <URL>.

In [274]:
len(kyt)

225

In [275]:
len(idx_color)

225

### bkk clean data

In [233]:
#df_bkk1 = load_obj(path_play + r"\translated_bkk_tweet1")
#df_bkk2 = load_obj(path_play + r"\translated_bkk_tweet2")

In [95]:
list_df = []
for filename in os.listdir(path_play+"\\bkk\\old"):
    with open(path_play + "\\bkk\\old\\" + filename, "rb") as f:
        in_path = path_play + "\\bkk\\old\\" + filename
        print(in_path)
        df = load_obj(in_path)
        df.reset_index(inplace=True, drop="index")
        list_df.append(df)

C:\Users\sociocom\Desktop\Crawler\backup\playing\bkk\old\translated_bkk_tweet1
C:\Users\sociocom\Desktop\Crawler\backup\playing\bkk\old\translated_bkk_tweet2
C:\Users\sociocom\Desktop\Crawler\backup\playing\bkk\old\translated_bkk_tweet3
C:\Users\sociocom\Desktop\Crawler\backup\playing\bkk\old\translated_bkk_tweet4


In [96]:
df_bkk_old = pd.concat(list_df)

In [97]:
list_df2 = []
for filename in os.listdir(path_play+"\\bkk\\new"):
    with open(path_play + "\\bkk\\new\\" + filename, "rb") as f:
        in_path = path_play + "\\bkk\\new\\" + filename
        print(in_path)
        df = load_obj(in_path)
        df.reset_index(inplace=True, drop="index")
        df.drop(columns=["text","place"], inplace=True)
        df.rename(columns={"text_trans":"text", "place_trans":"place"}, inplace=True)
        list_df2.append(df)

C:\Users\sociocom\Desktop\Crawler\backup\playing\bkk\new\bkk_translated_new_15-22June


In [98]:
df_bkk_new = pd.concat(list_df2)

In [99]:
df_bkk = pd.concat([df_bkk_old, df_bkk_new])
df_bkk.reset_index(inplace=True, drop="index")

In [100]:
df_bkk.describe()

,acc_id,acc,tweet_id,text,created_time,ex_lat,ex_long,place,lot_sw,lot_ne,tweet_reply_id,tweet_reply_user_id,tweet_reply_username,lang
count,181865,181865,181865,181865,181865,181865,181865,181865,181865,181865,181865,181865,181865,181865
unique,21060,21194,181865,156343,170429,7700,7686,258,210,208,42905,19685,19739,2
top,882432466526453760,404notdown,1008144896597770241,I do not know what to do.,2018-06-08 06:27:52,-1,-1,Thailand,"100.528375, 13.72951","100.541027, 13.749708",-1,-1,-1,th
freq,1161,1161,1,2218,5,159729,159729,12751,13623,13623,136652,134507,134507,149430


In [101]:
df_bkk = clean_place(df_bkk, "Myanmar")
df_bkk = clean_place(df_bkk, "Thailand")
df_bkk = clean_place(df_bkk, "Bangkok")

In [22]:
df_bkk.describe()

,acc_id,acc,tweet_id,text,created_time,ex_lat,ex_long,place,lot_sw,lot_ne,tweet_reply_id,tweet_reply_user_id,tweet_reply_username,lang
count,113992,113992,113992,113992,113992,113992,113992,113992,113992,113992,113992,113992,113992,113992
unique,14369,14431,113992,97663,107745,5307,5287,151,105,103,27705,13327,13345,2
top,882432466526453760,404notdown,1006255644804931586,I do not know what to do.,2018-06-08 06:27:52,-1,-1,Pathum Wan,"100.528375, 13.72951","100.541027, 13.749708",-1,-1,-1,th
freq,669,669,1,1506,5,100012,100012,6227,9837,9837,85289,84043,84043,96966


In [102]:
len(df_bkk[(df_bkk["ex_lat"]!= -1) & (df_bkk["tweet_reply_id"]!= "-1")])

39210

In [103]:
df_bkk.reset_index(inplace=True)

In [104]:
df_bkk["text"][1]

'Walk through (@Sam Market Market (Wat Phra Kaew Market) in Bang Rak, Bangkok) https://t.co/7TsvbPXdGE'

In [105]:
for i in range(len(df_bkk)):
    if (df_bkk["ex_lat"][i] == -1) | (df_bkk["tweet_reply_username"][i] == -1):
        continue
    idx_reply = df_bkk["text"][i].find(df_bkk["tweet_reply_username"][i])-1
    leng = len(df_bkk["tweet_reply_username"][i])+1
    df_bkk.at[i, "text"] = df_bkk["text"][i][:idx_reply] + " <MENTION> " + df_bkk["text"][i][idx_reply+leng:]

In [106]:
df_bkk.head()

,index,acc_id,acc,tweet_id,text,created_time,ex_lat,ex_long,place,lot_sw,lot_ne,tweet_reply_id,tweet_reply_user_id,tweet_reply_username,lang
0,0,137552452,ExpatDanBKK,1001726470962769920,"<MENTION> Hmmm, I suspect Bill Heinecke woul...",2018-05-30 07:26:16,13.742844,100.5682602,Vadhana,"100.55007, 13.732979","100.570733, 13.74842",1001725702192824321,710742220681293824,David_Aamez,en
1,1,372674276,NeungBellerin,1001726500725456897,Walk through (@Sam Market Market (Wat Phra Kae...,2018-05-30 07:26:23,13.724922,100.52291,Suriya Wong,"100.516253, 13.72236","100.536283, 13.730574",-1,-1,-1,th
2,2,1030302462,opalopongpang,1001726547160690689,I'm at Penguin Eat Shabu in Vadhana https://t....,2018-05-30 07:26:34,13.7388373,100.56067598,Vadhana,"100.55007, 13.732979","100.570733, 13.74842",-1,-1,-1,en
3,3,953277866661199872,Poppysuphannee,1001726564550168576,And it's not going to be a school ride. I do n...,2018-05-30 07:26:38,-1,-1,Frown,"100.469033, 13.773245","100.49389, 13.789877",-1,-1,-1,th
4,4,399872095,StoneSoupBKK,1001726646599041024,<MENTION> F@$king morons.,2018-05-30 07:26:58,-1,-1,Phra Khanong Nuea,"100.584345, 13.704628","100.606727, 13.729193",1001701922074701824,774174489458331649,TheMossadIL,en


In [110]:
bkk = separate_loc(df_bkk, bkk_lat, bkk_long)

In [111]:
idx_color["3"]

[(100.484605, 13.705779999999997),
 (100.49127166666666, 13.705779999999997),
 (100.484605, 13.712446666666663),
 (100.49127166666666, 13.712446666666663),
 '3']

In [112]:
count = 0
valid = 0
for i in range(len(bkk)):
    count += len(bkk[i])
    if len(bkk[i])>20:
        valid += 1
    print("#tweets in area",i,":" ,len(bkk[i]))
print(count)
print(valid)

#tweets in area 0 : 1
#tweets in area 1 : 0
#tweets in area 2 : 14
#tweets in area 3 : 12
#tweets in area 4 : 7
#tweets in area 5 : 46
#tweets in area 6 : 3
#tweets in area 7 : 7
#tweets in area 8 : 62
#tweets in area 9 : 41
#tweets in area 10 : 176
#tweets in area 11 : 32
#tweets in area 12 : 12
#tweets in area 13 : 16
#tweets in area 14 : 10
#tweets in area 15 : 0
#tweets in area 16 : 13
#tweets in area 17 : 32
#tweets in area 18 : 9
#tweets in area 19 : 0
#tweets in area 20 : 58
#tweets in area 21 : 29
#tweets in area 22 : 21
#tweets in area 23 : 79
#tweets in area 24 : 1745
#tweets in area 25 : 119
#tweets in area 26 : 124
#tweets in area 27 : 67
#tweets in area 28 : 4
#tweets in area 29 : 52
#tweets in area 30 : 5
#tweets in area 31 : 1
#tweets in area 32 : 72
#tweets in area 33 : 2
#tweets in area 34 : 3
#tweets in area 35 : 52
#tweets in area 36 : 9
#tweets in area 37 : 10
#tweets in area 38 : 24
#tweets in area 39 : 192
#tweets in area 40 : 328
#tweets in area 41 : 85
#tweets i

In [113]:
idx_color["56"] #Siam

[(100.50460499999998, 13.759113333333325),
 (100.51127166666664, 13.759113333333325),
 (100.50460499999998, 13.76577999999999),
 (100.51127166666664, 13.76577999999999),
 '56']

### prepare data

In [114]:
bkk_str = []
for y in bkk:
    doc = ""
    for x in y['text']:
        if "https://" in x.lower():
            idx = (x.lower().find("https://"))-1
            x = x[:idx] + " <URL>"
        if "I'm at " in x and "<URL>" in x:
            idx_i = (x.find("I'm at "))
            idx_url = (x.find("<URL>"))+5
            x = x[:idx_i]+x[idx_url:]+"<CHECK_IN>"
        doc += x + ". "
    doc += " \n"
    bkk_str.append(doc)

In [115]:
#for y in bkk_str:
#    pattern2.sub("<CHECK_IN>", y)
#    print(y)

In [116]:
print(bkk_str[25])

<CHECK_IN>. <CHECK_IN>. <CHECK_IN>. I have a pain in my stomach. 5555 (@ Bangkok) <URL>. <CHECK_IN>. Many @ jawwa @ (@ Bar Next Door w / @punnsann @ryongboslh) <URL>. <CHECK_IN>. <CHECK_IN>. <CHECK_IN>. <CHECK_IN>. This is a rock style ~ ~ (@ Bangkok) <URL>. (@ Bangkok) <URL>. <CHECK_IN>. I will not be able to do tha <MENTION>  will not be able to do that.. Come back !!! (@ Bangkok) <URL>. Halfway through, but check in first. Scared of Drunk (@Kraan Road in Phra Nakhon, Bangkok) <URL>.  (@ Bar Next Door) <URL>. <CHECK_IN>. <CHECK_IN>. <CHECK_IN>. <CHECK_IN>. More liver disease (@ Bangkok) <URL>. Life at the Hahaha (@ Bangkok) <URL>. Khon Kaen's nephew Eyes are broken for 2-3 days. Just have a chance to shoot (@ Bangkok) <URL>. <CHECK_IN>. <CHECK_IN>. Back to Dream World (@ Bangkok) <URL>. I'm not sure what to sa <MENTION> 'm not sure what to say.. Walking distance (@ Phuttasawat in Phra Nakhon, Bangkok) <URL>. Take a photo of yourself in @ (Bangkok City Pillar) in Phra Nakhon, Bangkok 

In [117]:
f = open(path_play + "/bkk_demo_train3","w",encoding="utf-8")
#file = open(path + "bkk_demo_test","w",encoding="utf-8")
for x in bkk_str:
    f.write(x)
    print(x)
f.close()

<CHECK_IN>.  

 

<CHECK_IN>. <CHECK_IN>. <CHECK_IN>. What do you eat? <URL>.  @ Riverside… <URL>. Rebirth leaw jaaa (@ Starbucks in Thon Buri, Bangkok) <URL>. After a 15 minute break ... <URL>. <CHECK_IN>. Start! (@ Starbucks in Thon Buri, Bangkok) <URL>. This is my first pos <MENTION> his is my first post.. <CHECK_IN>. <CHECK_IN>. <CHECK_IN>. <CHECK_IN>.  

<CHECK_IN>. <CHECK_IN>. <CHECK_IN>. <CHECK_IN>. The album is coming for a while ... <URL>. I'm sorry @ Buta Shubu Tha Bright <URL>. Whirlpool (@ Somdejprapinklao Hospital in Thon Buri, Bangkok) <URL>. # 13062018 @ My Home Flats ... <URL>. We do not sleep (OO). <URL>. My dog always make me smile  #15062018 @ My… <URL>. Goodnight my world and sweet dream … <URL>. We have a smaller waist than 1 inch .... <URL>.  

The first dental implant (@ dentist dentist) <URL>. Work with a gang with an apprentice. Same model (@ Bester Studio in Bangkok) <URL>. <CHECK_IN>. Today Friday ... <URL>. <CHECK_IN>. Do you like this? (@ Boom Shop in Bukkh

## Doc2Vec

In [276]:
train_corpus = list(read_corpus(path_play + "/kyt_demo_train3"))

In [277]:
model = gensim.models.doc2vec.Doc2Vec(vector_size=80, min_count=2, epochs=45)
#model = gensim.models.doc2vec.Doc2Vec(vector_size=80, min_count=2, epochs=100)

In [278]:
model.build_vocab(train_corpus)

In [279]:
#model.wv.vocab["สยาม"].count

In [280]:
%time model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)

Wall time: 8 s


In [281]:
ranks = []
second_ranks = []
all_sims = []
for doc_id in range(len(train_corpus)):
    inferred_vector = model.infer_vector(train_corpus[doc_id].words)
    
    sims = model.docvecs.most_similar([inferred_vector], topn=len(model.docvecs))
    all_sims.append(sims)
    
    rank = [docid for docid, sim in sims].index(doc_id)
    ranks.append(rank)
    
    second_ranks.append(sims[1])

In [282]:
#print(ranks,"\n")
print(ranks.count(0))

191


In [283]:
print('Document ({}): «{}»\n'.format(doc_id, ' '.join(train_corpus[doc_id].words)))
print(u'SIMILAR/DISSIMILAR DOCS PER MODEL %s:\n' % model)
for label, index in [('MOST', 0), ('MEDIAN', len(sims)//2), ('LEAST', len(sims) - 1)]:
    print(u'%s %s: «%s»\n' % (label, sims[index], ' '.join(train_corpus[sims[index][0]].words)))

Document (224): «lucky was canceled and the initial purpose ended shogakuin station eizandensha in kyoto city kyoto prefecture url check_in school road no ohara yamada farm tamago studio in sakyō kyoto url check_in check_in check_in check_in check_in check_in ramen all machines ramen ikedaya yokodera temple ikedaya for the first time in about half year ve been lucky to have no vacancies at noon sales the president soup feels like soy sauce is standing up with kirili pork is fatty but it is satisfied with small and large small pieces noodles have little stiffness and full it is wrong in tokyo url spring roll raw spring roll specialty shop spring roll stand wrap amp roll in sakyō kyoto url check_in check_in check_in check_in sad news during nap shogakuin station eizandensha in kyoto city kyoto prefecture url in kyoto at shogakuin superior noodles with soy sauce noodle yen and chashed increased yen while in an inconvenient place kyo url mention can not wait for my dream pochi_go it always

## Merge area

In [284]:
idx_color2 = copy.deepcopy(idx_color)

In [285]:
print_areas(idx_color)

 0  1  2  3  4  5  6  7  8  9  10  11  12  13  14 
15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 
30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 
45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 
60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 
75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 
90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 
105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 
120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 
135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 
150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 
165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 
180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 
195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 
210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 

 #areas : 225


{'0',
 '1',
 '10',
 '100',
 '101',
 '102',
 '103',
 '104',
 '105',
 '106',
 '107',
 '108',
 '109',
 '11',
 '110',
 '111',
 '112',
 '113',
 '114',
 '115',
 '116',
 '117',
 '118',
 '119',
 '12',
 '120',
 '121',
 '122',
 '123',
 '124',
 '125',
 '126',
 '127',
 '128',
 '129',
 '13',
 '130',
 '131',
 '132',
 '133',
 '134',
 '135',
 '136',
 '137',
 '138',
 '139',
 '14',
 '140',
 '141',
 '142',
 '143',
 '144',
 '145',
 '146',
 '147',
 '148',
 '149',
 '15',
 '150',
 '151',
 '152',
 '153',
 '154',
 '155',
 '156',
 '157',
 '158',
 '159',
 '16',
 '160',
 '161',
 '162',
 '163',
 '164',
 '165',
 '166',
 '167',
 '168',
 '169',
 '17',
 '170',
 '171',
 '172',
 '173',
 '174',
 '175',
 '176',
 '177',
 '178',
 '179',
 '18',
 '180',
 '181',
 '182',
 '183',
 '184',
 '185',
 '186',
 '187',
 '188',
 '189',
 '19',
 '190',
 '191',
 '192',
 '193',
 '194',
 '195',
 '196',
 '197',
 '198',
 '199',
 '2',
 '20',
 '200',
 '201',
 '202',
 '203',
 '204',
 '205',
 '206',
 '207',
 '208',
 '209',
 '21',
 '210',
 '211',
 '

In [291]:
#threshold = 0.7
area_limit = 32
areas = count_areas(idx_color2)
while areas > area_limit:
    idx_color2 = merge_area(threshold, idx_color2, kyt)
    threshold -= 0.001
    areas = count_areas(idx_color2)
    if (areas < 70): 
        print(areas)
print(threshold)

35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
3

In [292]:
print(threshold)
print(areas)

0.0008928999999993709
32


In [293]:
kyt_str[67]

'<CHECK_IN>. Tireless (@ Shin Fukutaro Tenjin River store in Kyoto City, Kyoto Prefecture) <URL>. <CHECK_IN>. culture. (@ cafe kamie in Kyoto, Kyoto) <URL>. <CHECK_IN>. From next week R. J. BeerZ ... <URL>. <CHECK_IN>. <CHECK_IN>. <CHECK_IN>. <CHECK_IN>. Chaahan and Chinese buckwheat tea (@ Shin Fukutanro Tenjin River store in Kyoto City, Kyoto prefecture) <URL>. <CHECK_IN>. At any time (@ Shin Fukutaro Tenjin River store in Kyoto City, Kyoto Prefecture) <URL>.  \n'

In [294]:
print_areas(idx_color2)

-1  1  1  1  1 -1 -1 -1  8  8  8 -1 -1 -1  14 
-1 -1 45  1 -1 45 45 -1 37 37 -1 -1 -1 -1 -1 
-1 45 45 -1 45 45 -1 37 37 37 -1 -1 42 -1 -1 
45 45 -1 45 45 45 -1 45 45 45 -1 -1 -1 -1 -1 
-1 75 75 75 75 75 75 75 75 75 75 75 -1 -1 -1 
75 75 75 75 75 75 75 75 75 75 75 75 -1 -1 -1 
-1 105 105 -1 -1 105 75 75 75 75 75 75 75 -1 -1 
105 105 105 105 105 105 111 105 113 75 75 75 75 75 75 
105 105 105 105 105 105 105 105 128 129 143 143 143 143 143 
105 105 105 105 139 105 141 142 143 143 143 143 143 143 143 
105 105 105 105 154 105 156 157 158 159 174 143 174 174 174 
105 105 105 105 105 105 105 172 173 174 174 174 174 174 174 
105 181 -1 181 105 105 105 187 188 174 174 191 105 174 174 
-1 181 181 181 -1 105 105 105 203 105 205 105 105 105 105 
-1 -1 -1 -1 214 -1 105 105 105 105 105 105 105 105 105 

 #areas : 32


{'-1',
 '1',
 '105',
 '111',
 '113',
 '128',
 '129',
 '139',
 '14',
 '141',
 '142',
 '143',
 '154',
 '156',
 '157',
 '158',
 '159',
 '172',
 '173',
 '174',
 '181',
 '187',
 '188',
 '191',
 '203',
 '205',
 '214',
 '37',
 '42',
 '45',
 '75',
 '8'}

In [295]:
kyt_all = [idx_color2,kyt,ranks,all_sims]

In [296]:
save_obj(path_play + r"\kyt_str", kyt_str)

In [297]:
save_obj(path_play + r"\kyt_data2", kyt_all)

In [298]:
print_volumn(kyt)

  0  15  35  14  11   0   0   4 275 187  16   1   0   0  11 
 16   1   0   0  11   2   1  11  20   5  27  48   5  18  41 
 27  48   5  18  41   6   1   2   0   0   0  12 162   7  28 
  0  12 162   7  28  24   2  27  16  20   2   9 165   0   0 
  2   9 165   0   0  11  88   5 228  20  22   4  29  26  38 
 22   4  29  26  38   6   4   0   0   0   6 161  13  22  15 
  6 161  13  22  15 119  34  13  31  40  13  35   1   2   0 
 13  35   1   2   0  28  25  11  13  29 261  61 205  77  58 
261  61 205  77  58  13  45   0   2   1   2  22  49   5   4 
  2  22  49   5   4  61 272 146  28  92  88 114 147   0   4 
 88 114 147   0   4  15  28  31 266  35 148 879 218 2500 186 
148 879 218 2500 186  90 136  88  43  20  80  87  48  31 259 
 80  87  48  31 259 101 199 111 726 425 192  76  64  49  67 
192  76  64  49  67  87  55  86 151 625 124 2468 393 229  74 
124 2468 393 229  74 118  94  55  85  38 273  87  45  98 7457 


In [249]:
bkk_all = [idx_color2, bkk, ranks, all_sims]

In [250]:
save_obj(path_play + r"\bkk_str", bkk_str)

In [251]:
save_obj(path_play + r"\bkk_data2", bkk_all)

In [253]:
print_volumn(bkk)

  1   0  14  12   7  46   3   7  62  41 176  32  12  16  10 
176  32  12  16  10   0  13  32   9   0  58  29  21  79 1745 
 58  29  21  79 1745 119 124  67   4  52   5   1  72   2   3 
  5   1  72   2   3  52   9  10  24 192 328  85  35   2  20 
328  85  35   2  20   1   9   2  23  14  12  31  52 222  19 
 12  31  52 222  19  47  16  15  74  22   5  13  29  16  56 
  5  13  29  16  56 126  85  71  78  40   6  12  16  11  18 
  6  12  16  11  18   1   2   3  14  40 112  39  28  77  27 
112  39  28  77  27   3   8   4  10  28   9   1   4  22  20 
  9   1   4  22  20 292 120 209 297 264  30  52 117   5   2 
 30  52 117   5   2   3  81  30  25  26  66 297 104 1552 1958 
 66 297 104 1552 1958 152 168  53  25  30   1  14  29  51  30 
  1  14  29  51  30  48 187  26 434 954  65 287  42  86  94 
 65 287  42  86  94  14   6   2   3   9  42  27  72 315  12 
 42  27  72 315  12   1  10  30   7  96   1   0   0   0  17 


In [252]:
print_areas(idx_color2)

-1 -1 16 16 -1  5 -1 -1  5  5  5 25 25  5  5 
-1 16 16 -1 -1  5  5  5  5 24 25 25 25 -1  5 
-1 -1 16 -1 -1 61 -1 61  5  5 40  5  5 -1  5 
-1 -1 -1 61 61 61 61 61 53 61  5  5  5  5  5 
-1 61 61 61 61 61 61 61 61 61 -1  5  5  5  5 
-1 -1 -1 61 61 61 61 61 61 61 -1 -1 -1  5  5 
-1 -1 -1 106 106 95 61 61 98 99 100 100 100 -1 -1 
-1 106 106 106 106 106 111 106 113 114 100 100 100 100 100 
-1 106 106 106 106 106 106 106 128 129 100 131 146 100 100 
135 -1 -1 -1 -1 154 106 106 143 106 -1 146 146 -1 100 
-1 -1 -1 -1 154 154 -1 106 106 106 -1 146 146 146 -1 
-1 -1 -1 -1 -1 184 154 172 173 174 174 -1 -1 146 146 
-1 -1 -1 -1 184 184 186 184 184 184 184 184 192 146 -1 
-1 -1 -1 -1 213 213 213 184 213 213 184 184 207 146 146 
-1 -1 -1 213 213 213 213 213 213 213 -1 -1 222 -1 146 

 #areas : 32


{'-1',
 '100',
 '106',
 '111',
 '113',
 '114',
 '128',
 '129',
 '131',
 '135',
 '143',
 '146',
 '154',
 '16',
 '172',
 '173',
 '174',
 '184',
 '186',
 '192',
 '207',
 '213',
 '222',
 '24',
 '25',
 '40',
 '5',
 '53',
 '61',
 '95',
 '98',
 '99'}

In [142]:
print(str(list(idx_color["0"][0])))

[135.6753, 34.955205]


In [299]:
map_color = {}
#color = ['#17263c','#c9b2a6','#dcd2be','#ae9e90','#dfd2ae','#dfd2ae', '#93817c','#f8c967','#8f7d77','#dfd2ae','#b9d3c2']
color = ['#56000a','orange','gray','black','brown','pink', 'purple','yellow','red','green','blue','#001994','#69002c','#936c6c','#76539f','#005360',
        '#0d92e8','#98fefb','#e552e5','#fd130c','#660585','#754cda','#cadb11','#7668bb','#62ab12','#cadb11','#772838','#c0b018',
         '#b8bca8','#fd5620','#635782','#3e520a','#2c63c2','#7be491','#7a1154','#b45a30','#9270d8','#d0dc62','#693487','#dbb8df']

In [300]:
for i in range(areas):
    map_color[str(i)]=color[i%len(color)]

In [301]:
map_color

{'0': '#56000a',
 '1': 'orange',
 '10': 'blue',
 '11': '#001994',
 '12': '#69002c',
 '13': '#936c6c',
 '14': '#76539f',
 '15': '#005360',
 '16': '#0d92e8',
 '17': '#98fefb',
 '18': '#e552e5',
 '19': '#fd130c',
 '2': 'gray',
 '20': '#660585',
 '21': '#754cda',
 '22': '#cadb11',
 '23': '#7668bb',
 '24': '#62ab12',
 '25': '#cadb11',
 '26': '#772838',
 '27': '#c0b018',
 '28': '#b8bca8',
 '29': '#fd5620',
 '3': 'black',
 '30': '#635782',
 '31': '#3e520a',
 '4': 'brown',
 '5': 'pink',
 '6': 'purple',
 '7': 'yellow',
 '8': 'red',
 '9': 'green'}

In [441]:
idx_color2["0"]

[(100.484605, 13.68578),
 (100.494605, 13.68578),
 (100.484605, 13.69578),
 (100.494605, 13.69578),
 '-1']

In [377]:
f_loc = open("latlong_kyt_225_old.txt","w", encoding="utf-8")
color_area = list(print_areas(idx_color2))
color_area.sort()
f_loc.write("""{
  "type": "FeatureCollection",
  "features": [
  """)
loop = grid_size*grid_size
for i in range(loop):
    idx = color_area.index(idx_color2[str(i)][4])
    colorful = map_color[str(idx)]
    #print(colorful)
    #if colorful != "#56000a":
    latlong = ""
    #for j in range(4):
    #    latlong += str(list(idx_color2[str(i)][j])) +", "
    latlong += str(list(idx_color2[str(i)][0])) +", "
    latlong += str(list(idx_color2[str(i)][1])) +", "
    latlong += str(list(idx_color2[str(i)][3])) +", "
    latlong += str(list(idx_color2[str(i)][2])) +", "
    latlong += str(list(idx_color2[str(i)][0]))
    #witf.write("{location: new google.maps.LatLng(%f, %f)},\n" % (test_kyo1['ex_lat'][i],test_kyo1['ex_long'][i]))
    f_loc.write("""{
      "type": "Feature",
      "properties": {
        "letter": "o",
        "color": "%s",
        "rank": "15",
        "ascii": "111"
      },
      "geometry": {
        "type": "Polygon",
        "coordinates": [
          [
            %s 
          ]
        ]
      }
    },
    """ % (colorful, latlong))
    # remember to delete last ","
f_loc.write("""]
}""")
f_loc.close()


-1 -1  2 -1 -1 -1 -1 -1  8  8 -1 -1 -1 -1 -1 
-1 -1 -1 -1 -1 34 34 -1 -1  8 -1 -1 -1 -1 -1 
-1 -1 32 -1 34 34 -1 37 -1 -1 -1 -1 42 -1 -1 
-1 46 -1 48 -1 34 -1 37 37 37 -1 -1 -1 -1 -1 
-1 75 -1 48 -1 79 79 -1 79 79 -1 71 -1 -1 -1 
75 75 -1 -1 79 79 79 79 79 79 -1 79 -1 -1 -1 
-1 75 75 -1 -1 75 79 79 79 79 79 79 79 -1 -1 
-1 120 120 120 120 120 111 79 113 79 79 79 79 79 -1 
120 120 120 120 120 120 120 120 128 129 143 143 143 143 143 
120 120 120 120 139 120 141 142 143 143 143 143 143 143 120 
120 120 120 120 154 120 120 157 158 159 174 143 174 174 -1 
120 120 120 120 120 120 171 172 173 174 174 174 174 174 174 
120 181 -1 120 120 120 120 187 188 174 174 191 206 174 174 
-1 181 -1 -1 -1 120 120 120 203 120 205 206 120 120 120 
-1 -1 -1 -1 -1 -1 -1 120 120 120 120 120 120 120 120 

 #areas : 36


In [302]:
f_loc = open("latlong_kyt_225.txt","w", encoding="utf-8")
color_area = list(print_areas(idx_color2))
color_area.sort()
f_loc.write("""{
  "type": "FeatureCollection",
  "features": [
  """)
loop = grid_size*grid_size
for i in range(loop):
    idx = color_area.index(idx_color2[str(i)][4])
    colorful = map_color[str(idx)]
    #print(colorful)
    if colorful != "#56000a":
        latlong = ""
        #for j in range(4):
        #    latlong += str(list(idx_color2[str(i)][j])) +", "
        latlong += str(list(idx_color2[str(i)][0])) +", "
        latlong += str(list(idx_color2[str(i)][1])) +", "
        latlong += str(list(idx_color2[str(i)][3])) +", "
        latlong += str(list(idx_color2[str(i)][2])) +", "
        latlong += str(list(idx_color2[str(i)][0]))
        #witf.write("{location: new google.maps.LatLng(%f, %f)},\n" % (test_kyo1['ex_lat'][i],test_kyo1['ex_long'][i]))
        f_loc.write("""{
          "type": "Feature",
          "properties": {
            "letter": "o",
            "color": "%s",
            "rank": "15",
            "ascii": "111"
          },
          "geometry": {
            "type": "Polygon",
            "coordinates": [
              [
                %s 
              ]
            ]
          }
        },
        """ % (colorful, latlong))
    # remember to delete last ","
f_loc.write("""]
}""")
f_loc.close()


-1  1  1  1  1 -1 -1 -1  8  8  8 -1 -1 -1  14 
-1 -1 45  1 -1 45 45 -1 37 37 -1 -1 -1 -1 -1 
-1 45 45 -1 45 45 -1 37 37 37 -1 -1 42 -1 -1 
45 45 -1 45 45 45 -1 45 45 45 -1 -1 -1 -1 -1 
-1 75 75 75 75 75 75 75 75 75 75 75 -1 -1 -1 
75 75 75 75 75 75 75 75 75 75 75 75 -1 -1 -1 
-1 105 105 -1 -1 105 75 75 75 75 75 75 75 -1 -1 
105 105 105 105 105 105 111 105 113 75 75 75 75 75 75 
105 105 105 105 105 105 105 105 128 129 143 143 143 143 143 
105 105 105 105 139 105 141 142 143 143 143 143 143 143 143 
105 105 105 105 154 105 156 157 158 159 174 143 174 174 174 
105 105 105 105 105 105 105 172 173 174 174 174 174 174 174 
105 181 -1 181 105 105 105 187 188 174 174 191 105 174 174 
-1 181 181 181 -1 105 105 105 203 105 205 105 105 105 105 
-1 -1 -1 -1 214 -1 105 105 105 105 105 105 105 105 105 

 #areas : 32
